In [13]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,Lambda,add,Reshape,concatenate, BatchNormalization,GlobalAveragePooling2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.regularizers import l2
%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

In [14]:
PATH = os.getcwd()

In [15]:
data_path = PATH + '/FULL_FACE_TRAIN'

In [16]:
data_dir_list = os.listdir(data_path)

In [17]:
img_data_list=[]

In [18]:
for dataset in data_dir_list:
	img_list=os.listdir(data_path+'/'+ dataset)
	print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
	for img in img_list:
		img_path = data_path + '/'+ dataset + '/'+ img
		img = image.load_img(img_path, target_size=(224, 224))
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
#		x = x/255
		#print('Input image shape:', x.shape)
		img_data_list.append(x)   

Loaded the images of dataset-Surprise face cropped

Loaded the images of dataset-Disgust face cropped

Loaded the images of dataset-Happy face cropped

Loaded the images of dataset-Contempt face cropped

Loaded the images of dataset-Angry face cropped

Loaded the images of dataset-Sad face cropped

Loaded the images of dataset-Fear face cropped



In [19]:
img_data = np.array(img_data_list) # takes time"
        
        # To bring data in the form (no. of images, row, col, channels)
        #print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
img_data=img_data[0]
       # print (img_data.shape) 

In [20]:
num_classes = 7
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

In [21]:
labels[0:540]=0
labels[540:880]=1
labels[880:1495]=2
labels[1495:1611]=3
labels[1611:2043]=4
labels[2043:2363]=5
labels[2363:2600]=6

In [22]:
names = ['Surprised face','Sad face','Happy face','Fear face','Disgust face','Contempt face','Angry face']

In [25]:
from keras.utils import np_utils
Y = np_utils.to_categorical(labels, num_classes)

In [28]:
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle
x,y = shuffle(img_data,Y, random_state=2) # TAKES TIME 
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2) 

/Users/hardika/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [34]:
def grouped_convolution_block(input1, grouped_channels, cardinality, strides):
    init = input1
    group_list = []
    for c in range(cardinality):
        x = Lambda(lambda z: z[:, :, :, c * grouped_channels:(c + 1) * grouped_channels])(input1)
        x = Conv2D(grouped_channels, (3, 3), padding='same', use_bias=False, strides=(strides, strides), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(x)
        group_list.append(x)
    group_merge = concatenate(group_list, axis=-1)
    x = BatchNormalization(axis=-1)(group_merge)
    x = Activation('relu')(x)
    return x

def se_bottleneck_block(input1, filters, cardinality=32, strides=1, r=16):
    init = input1

    grouped_channels = int(filters / cardinality)
    if init._keras_shape[-1] != 2 * filters:
        init = Conv2D(filters * 2, (1, 1), padding='same', strides=(strides, strides),use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(init)
        init = BatchNormalization(axis=-1)(init)

    X = Conv2D(filters, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(input1)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)

    X = grouped_convolution_block(X, grouped_channels, cardinality, strides)

    X = Conv2D(filters * 2, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X)
    X = BatchNormalization(axis=-1)(X)

    X_increase = X
    X = GlobalAveragePooling2D(data_format='channels_last')(X)
    X = Reshape([1,1,filters * 2])(X)
    X = Conv2D(filters = filters * 2 // r, kernel_size = (1, 1))(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = filters * 2, kernel_size = (1, 1))(X)
    X = Activation('sigmoid')(X)
    X = layers.add([keras.layers.multiply([X,X_increase]),init])
    X = Activation('relu')(X)
    return X

#ResNeXt 50: depth = [3,4,6,3]
#ResNeXt 101: depth =[3,4,23,3]
#ResNeXt 152: depth =[3,8,36,3]
def SE_ResNeXt(input_shape = (224, 224,3), depth=[3,8,36,3], cardinality=32, width=4, r=16):
    X_input = Input(input_shape)
    X = Conv2D(64, (7, 7), padding='same', use_bias=False, strides=(2, 2), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X_input)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(X)
    
    filters = cardinality * width
    filters_list = []
    for i in range(len(depth)):
        filters_list.append(filters)
        filters *= 2 

    for i in range(depth[0]):
        X = se_bottleneck_block(X, filters_list[0], cardinality, strides=1, r=r)

    depth = depth[1:]  
    filters_list = filters_list[1:]  

    for block_idx, n_i in enumerate(depth):
        for i in range(n_i):
            if i == 0:
                X = se_bottleneck_block(X, filters_list[block_idx], cardinality, strides=2, r=r)
            else:
                X = se_bottleneck_block(X, filters_list[block_idx], cardinality, strides=1, r=r)
    X = GlobalAveragePooling2D()(X)
    X = Dense(7, use_bias=False, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='SE-ResNeXt')

    return model

In [35]:
model = SE_ResNeXt()

In [36]:

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1806 (Conv2D)            (None, 112, 112, 64) 9408        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_156 (BatchN (None, 112, 112, 64) 256         conv2d_1806[0][0]                
__________________________________________________________________________________________________
activation_252 (Activation)     (None, 112, 112, 64) 0           batch_normalization_156[0][0]    
__________________________________________________________________________________________________
max_poolin

__________________________________________________________________________________________________
conv2d_2065 (Conv2D)            (None, 28, 28, 8)    576         lambda_1828[0][0]                
__________________________________________________________________________________________________
conv2d_2066 (Conv2D)            (None, 28, 28, 8)    576         lambda_1829[0][0]                
__________________________________________________________________________________________________
conv2d_2067 (Conv2D)            (None, 28, 28, 8)    576         lambda_1830[0][0]                
__________________________________________________________________________________________________
conv2d_2068 (Conv2D)            (None, 28, 28, 8)    576         lambda_1831[0][0]                
__________________________________________________________________________________________________
conv2d_2069 (Conv2D)            (None, 28, 28, 8)    576         lambda_1832[0][0]                
__________

__________________________________________________________________________________________________
lambda_2084 (Lambda)            (None, 14, 14, 16)   0           activation_328[0][0]             
__________________________________________________________________________________________________
lambda_2085 (Lambda)            (None, 14, 14, 16)   0           activation_328[0][0]             
__________________________________________________________________________________________________
lambda_2086 (Lambda)            (None, 14, 14, 16)   0           activation_328[0][0]             
__________________________________________________________________________________________________
lambda_2087 (Lambda)            (None, 14, 14, 16)   0           activation_328[0][0]             
__________________________________________________________________________________________________
lambda_2088 (Lambda)            (None, 14, 14, 16)   0           activation_328[0][0]             
__________

                                                                 conv2d_2614[0][0]                
                                                                 conv2d_2615[0][0]                
                                                                 conv2d_2616[0][0]                
                                                                 conv2d_2617[0][0]                
                                                                 conv2d_2618[0][0]                
                                                                 conv2d_2619[0][0]                
                                                                 conv2d_2620[0][0]                
                                                                 conv2d_2621[0][0]                
                                                                 conv2d_2622[0][0]                
                                                                 conv2d_2623[0][0]                
          

conv2d_2934 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2600[0][0]                
__________________________________________________________________________________________________
conv2d_2935 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2601[0][0]                
__________________________________________________________________________________________________
conv2d_2936 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2602[0][0]                
__________________________________________________________________________________________________
conv2d_2937 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2603[0][0]                
__________________________________________________________________________________________________
conv2d_2938 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2604[0][0]                
__________________________________________________________________________________________________
conv2d_293

conv2d_3238 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2872[0][0]                
__________________________________________________________________________________________________
conv2d_3239 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2873[0][0]                
__________________________________________________________________________________________________
conv2d_3240 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2874[0][0]                
__________________________________________________________________________________________________
conv2d_3241 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2875[0][0]                
__________________________________________________________________________________________________
conv2d_3242 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2876[0][0]                
__________________________________________________________________________________________________
conv2d_324

conv2d_3540 (Conv2D)            (None, 7, 7, 32)     9216        lambda_3137[0][0]                
__________________________________________________________________________________________________
conv2d_3541 (Conv2D)            (None, 7, 7, 32)     9216        lambda_3138[0][0]                
__________________________________________________________________________________________________
conv2d_3542 (Conv2D)            (None, 7, 7, 32)     9216        lambda_3139[0][0]                
__________________________________________________________________________________________________
conv2d_3543 (Conv2D)            (None, 7, 7, 32)     9216        lambda_3140[0][0]                
__________________________________________________________________________________________________
conv2d_3544 (Conv2D)            (None, 7, 7, 32)     9216        lambda_3141[0][0]                
__________________________________________________________________________________________________
conv2d_354

In [37]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
hist = model.fit(X_train, y_train,epochs = 1, batch_size=32,validation_data=(X_test,y_test), shuffle = True)

Train on 1820 samples, validate on 780 samples
Epoch 1/1
  96/1820 [>.............................] - ETA: 2:31:59 - loss: 102.9737 - acc: 0.2604

KeyboardInterrupt: 